In [1]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.9 MB/s eta 0:00:00


In [2]:
import torch
from datasets import load_dataset
from transformers import (
    PegasusTokenizer,
    PegasusForConditionalGeneration,
    Trainer,
    TrainingArguments
)
import evaluate


In [3]:
dataset = load_dataset("cnn_dailymail", "3.0.0")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

3.0.0/train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

3.0.0/train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

3.0.0/train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

3.0.0/validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

3.0.0/test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [4]:
model_name = "google/pegasus-arxiv"

tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name)
model.gradient_checkpointing_enable()

tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-arxiv and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [5]:
def preprocess(batch):
    inputs = tokenizer(
        batch["article"],
        truncation=True,
        padding="max_length",
        max_length=384   # ⬅️ OBLIGATOIRE si 5000
    )

    targets = tokenizer(
        batch["highlights"],
        truncation=True,
        padding="max_length",
        max_length=96    # ⬅️ OBLIGATOIRE si 5000
    )

    inputs["labels"] = targets["input_ids"]
    return inputs

In [6]:
tokenized_datasets = dataset.map(
    preprocess,
    batched=True,
    remove_columns=dataset["train"].column_names
)


Map:   0%|          | 0/287113 [00:00<?, ? examples/s]

Map:   0%|          | 0/13368 [00:00<?, ? examples/s]

Map:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [7]:
train_subset = tokenized_datasets["train"].shuffle(seed=42).select(range(7000))
val_subset   = tokenized_datasets["validation"].select(range(1400))
test_subset  = tokenized_datasets["test"].select(range(1400))


In [8]:
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="steps",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=4,   # ⬅️ réduit
    fp16=True,
    learning_rate=2e-5,
    num_train_epochs=5,              # ⬅️ suffisant
    logging_steps=100,
    save_steps=500,
    save_total_limit=5,
    report_to="none"
)

In [9]:
!pip install rouge_score
rouge = evaluate.load("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred

    decoded_preds = tokenizer.batch_decode(
        predictions, skip_special_tokens=True
    )
    decoded_labels = tokenizer.batch_decode(
        labels, skip_special_tokens=True
    )

    result = rouge.compute(
        predictions=decoded_preds,
        references=decoded_labels
    )

    return {
        "rouge1": result["rouge1"],
        "rouge2": result["rouge2"],
        "rougeL": result["rougeL"],
        "rougeLsum": result["rougeLsum"]
    }

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=f3833580f43b1352ec99af9646798a4d21c21652fbec57f620eecefe7df8a58b
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge_score


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_subset,
    eval_dataset=val_subset,
    tokenizer=tokenizer
)

# Reduce the maximum length for generation during evaluation to mitigate OutOfMemoryError
# The default generation_max_length is often 128 for Pegasus-CNN_Dailymail.
# Reducing it to 64 should help with memory without severely impacting summary quality.

trainer.train()


/tmp/ipython-input-1952271151.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None}.


Step,Training Loss,Validation Loss
100,7.966500,7.949341
200,7.068500,7.238331
300,6.459400,5.737519
400,5.072900,2.986656
500,4.093800,2.582783
600,3.680300,2.478449
700,3.417600,2.379769
800,3.361900,2.318345
900,3.289100,2.283835
1000,3.158800,2.233611


/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:3918: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 256, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


Step,Training Loss,Validation Loss
100,7.966500,7.949341
200,7.068500,7.238331
300,6.459400,5.737519
400,5.072900,2.986656
500,4.093800,2.582783
600,3.680300,2.478449
700,3.417600,2.379769
800,3.361900,2.318345
900,3.289100,2.283835
1000,3.158800,2.233611


In [ ]:
import torch

def evaluate_rouge_manually(dataset, model, tokenizer, max_samples=200):
    model.eval()
    preds, refs = [], []

    for i in range(max_samples):
        example = dataset[i]

        input_ids = torch.tensor(example["input_ids"]).unsqueeze(0).to(model.device)
        attention_mask = torch.tensor(example["attention_mask"]).unsqueeze(0).to(model.device)

        with torch.no_grad():
            summary_ids = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_length=64,
                num_beams=4
            )

        preds.append(tokenizer.decode(summary_ids[0], skip_special_tokens=True))
        refs.append(tokenizer.decode(example["labels"], skip_special_tokens=True))

        del input_ids, attention_mask, summary_ids
        torch.cuda.empty_cache()

    return preds, refs


In [ ]:
predictions, references = evaluate_rouge_manually(
    test_subset,
    model,
    tokenizer,
    max_samples=200
)

results = rouge.compute(
    predictions=predictions,
    references=references
)

print(results)
